In [1]:
#Joint Annotation 만드는 ipynb file

In [2]:
import cv2
import tensorflow as tf
import os
import numpy as np
import network as net
import json
from Training import openpose
import matplotlib.pyplot as plt
from scipy.ndimage.filters import maximum_filter
import math
from collections import Counter, defaultdict
import itertools
import matplotlib as mpl
import time
import ast
from scipy.ndimage.filters import gaussian_filter
import matplotlib

C:\Users\JAEKYU\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Open Pose Class Loading Finish


In [3]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [4]:
def make_test_batch(img_path, batch_size):
    num_of_data = len(img_path)
    index = np.arange(0, num_of_data)
    np.random.shuffle(index)
    index = index[:batch_size]
    
    shuffled_img_data = [img_path[i] for i in index]
    
    return np.asarray(shuffled_img_data)

def path_to_image(img_path, batch_size):
    #buffer 선언
    image_data = np.zeros((batch_size, 356, 356, 3), np.uint8)
    
    index = 0
    for img in (img_path):
        image_data[index] = cv2.imread(img)
        index = index + 1;

        
        
    return image_data

def load_test_data(save_path):
    img_path = save_path#"./MPII_Dataset/resized_test_image/"
    file_path = []
    file_list = os.listdir(img_path)
    for i in (file_list):
        file_path.append(img_path + i)
    return file_path

def padRightDownCorner(img, stride, padValue):
    h = img.shape[0]
    w = img.shape[1]

    pad = 4 * [None]
    pad[0] = 0 # up
    pad[1] = 0 # left
    pad[2] = 0 if (h%stride==0) else stride - (h % stride) # down
    pad[3] = 0 if (w%stride==0) else stride - (w % stride) # right

    img_padded = img
    pad_up = np.tile(img_padded[0:1,:,:]*0 + padValue, (pad[0], 1, 1))
    img_padded = np.concatenate((pad_up, img_padded), axis=0)
    pad_left = np.tile(img_padded[:,0:1,:]*0 + padValue, (1, pad[1], 1))
    img_padded = np.concatenate((pad_left, img_padded), axis=1)
    pad_down = np.tile(img_padded[-2:-1,:,:]*0 + padValue, (pad[2], 1, 1))
    img_padded = np.concatenate((img_padded, pad_down), axis=0)
    pad_right = np.tile(img_padded[:,-2:-1,:]*0 + padValue, (1, pad[3], 1))
    img_padded = np.concatenate((img_padded, pad_right), axis=1)

    return img_padded, pad

In [5]:
#f = open("./Video_annotation/1.txt", 'w')

def demo_view(oriImg, input_heatmap, input_paf):
    nms_joint = []
    #input_heatmap = np.squeeze(input_heatmap)
    heatmap_avg = np.zeros((oriImg.shape[0], oriImg.shape[1], 17))
    paf_avg = np.zeros((oriImg.shape[0], oriImg.shape[1], 34))

    
    heatmap_avg = heatmap_avg + input_heatmap
    paf_avg = paf_avg + input_paf
    all_peaks = []
    peak_counter = 0

    for part in range(17-1):
        map_ori = heatmap_avg[:,:,part]
        map = gaussian_filter(map_ori, sigma=3)

        map_left = np.zeros(map.shape)
        #print(np.shape(map_left))
        map_left[1:,:] = map[:-1,:]
        map_right = np.zeros(map.shape)
        map_right[:-1,:] = map[1:,:]
        map_up = np.zeros(map.shape)
        map_up[:,1:] = map[:,:-1]
        map_down = np.zeros(map.shape)
        map_down[:,:-1] = map[:,1:]

        peaks_binary = np.logical_and.reduce((map>=map_left, map>=map_right, map>=map_up,\
                                              map>=map_down, map > 0.1))
        peaks = list(zip(np.nonzero(peaks_binary)[1], np.nonzero(peaks_binary)[0])) # note reverse
        peaks_with_score = [x + (map_ori[x[1],x[0]],) for x in peaks]
        id = range(peak_counter, peak_counter + len(peaks))
        peaks_with_score_and_id = [peaks_with_score[i] + (id[i],) for i in range(len(id))]

        all_peaks.append(peaks_with_score_and_id)
        peak_counter += len(peaks)

    #print(all_peaks)########################################################################################
    nms_joint.append(all_peaks)
    #all_peaks_file = ",".join(str(x) for x in all_peaks)
    #f.write(all_peaks_file)
    #f.write("\n")
    

    return nms_joint

In [6]:
limbSeq = [(9, 8), (8, 13), (8, 12), (8, 7), (12, 11), (11, 10), (13, 14), (14, 15), \
           (7, 6), (6, 3),  (3, 4),  (4, 5), (6, 2),   (2, 1),   (1, 0)]#16개 조인트, 15개 limb

mapIdx = [(16, 17), (18, 19), (20, 21), (22, 23), (28, 29), (30, 31), (24, 25), (26, 27), (32, 33), \
          (34, 35), (38, 39), (40, 41), (36, 37), (42, 43), (44, 45)]#x, y vectormap Pairs 24,25

colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
          [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
          [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85]]
img_width = 356
img_data_path = "./video_with_abstacle/TrainingSet_2/test/label_image/"
img_list = os.listdir(img_data_path)
img_filename_list = os.listdir(img_data_path)
for i, img in enumerate(img_list):
    img_list[i] = img_data_path + img
with tf.Session() as sess:
#with tf.Session(config=tf.ConfigProto(device_count={'GPU': 0})) as sess:
    obj = openpose(batch_size=1, sess = sess)
    sess.run(tf.global_variables_initializer())
    SAVE_PATH = "C:/Users/JAEKYU/Documents/Jupyter Notebook/Open_Pose/Weight/Weight.ckpt"
    saver = tf.train.Saver()
    saver.restore(sess, SAVE_PATH)
    for index, img in enumerate(img_list):
        #cap = cv2.VideoCapture(0)
        img_data = cv2.imread(img)
        heatmap, vectormap = obj.demo_test([img_data])
        heatmap = np.squeeze(heatmap)
        vectormap = np.squeeze(vectormap)
        resized_heatmap = np.zeros((17, 356, 356))
        resized_vectormap = np.zeros((34, 356, 356))

        heatmap = np.transpose(heatmap, (2,0, 1))
        vectormap = np.transpose(vectormap, (2,0, 1))

        for k in range(17):
            resized_heatmap[k] = cv2.resize(heatmap[k], (356, 356))
        for l in range(34):
            resized_vectormap[l] = cv2.resize(vectormap[l], (356, 356))

        resized_heatmap = np.transpose(resized_heatmap, (1,2, 0))
        resized_vectormap = np.transpose(resized_vectormap, (1,2, 0))

        #output, nms_joint = demo_view(img_color, resized_heatmap, resized_vectormap)
        nms_joint = demo_view(img_data, resized_heatmap, resized_vectormap)
        json_nms_joint = dict()
        #print(img_filename_list[index])
        with open("./video_with_abstacle/TrainingSet_2/test/img_anno/" + img_filename_list[index][0:-4] + '.json', 'w') as f:
            for j in range(16):
                if(len(nms_joint[0][j]) == 0):
                    pass
                else:
                    json_nms_joint['%d'%j] = nms_joint[0][j][0][0:2]
            print(json_nms_joint)
            dumped = json.dumps(json_nms_joint, cls=NumpyEncoder)

            json.dump(dumped, f)
            f.write('\n')

            json_nms_joint.clear()
            print("------------------------------------")
        f.close()
            

test data load start
test data load finish
open pose init complete
INFO:tensorflow:Restoring parameters from C:/Users/JAEKYU/Documents/Jupyter Notebook/Open_Pose/Weight/Weight.ckpt
{'0': (181, 302), '1': (165, 246), '2': (190, 198), '3': (214, 190), '4': (214, 246), '5': (214, 295), '6': (206, 190), '7': (175, 149), '8': (173, 133), '9': (173, 92), '10': (125, 165), '11': (146, 173), '12': (157, 149), '13': (190, 133), '14': (214, 149), '15': (222, 165)}
------------------------------------
{'0': (173, 303), '1': (165, 246), '2': (190, 205), '3': (206, 190), '4': (214, 246), '5': (214, 295), '6': (198, 205), '7': (173, 149), '8': (173, 141), '9': (173, 93), '10': (125, 144), '11': (141, 165), '12': (157, 149), '13': (198, 149), '14': (214, 173), '15': (222, 190)}
------------------------------------
{'0': (149, 303), '1': (149, 246), '2': (149, 182), '3': (173, 187), '4': (173, 254), '5': (165, 305), '6': (165, 182), '7': (165, 117), '8': (165, 120), '9': (165, 76), '10': (109, 173), '

{'0': (149, 295), '1': (157, 246), '2': (157, 182), '3': (182, 174), '4': (190, 238), '5': (190, 295), '6': (173, 182), '7': (165, 117), '8': (165, 117), '9': (165, 76), '10': (125, 190), '11': (133, 157), '12': (149, 117), '13': (182, 117), '14': (198, 140), '15': (214, 133)}
------------------------------------
{'0': (149, 303), '1': (157, 246), '2': (157, 182), '3': (182, 181), '4': (182, 246), '5': (182, 295), '6': (165, 182), '7': (165, 125), '8': (165, 117), '9': (165, 76), '10': (117, 190), '11': (125, 165), '12': (149, 125), '13': (187, 117), '14': (206, 125), '15': (213, 118)}
------------------------------------
{'0': (149, 303), '1': (157, 246), '2': (157, 182), '3': (173, 182), '4': (182, 246), '5': (173, 303), '6': (165, 182), '7': (165, 125), '8': (165, 117), '9': (165, 76), '10': (109, 182), '12': (141, 141), '13': (182, 117), '14': (198, 125), '15': (206, 125)}
------------------------------------


In [7]:
nms_joint

[[[(157, 301, 0.35670003294944763, 0)],
  [(173, 254, 0.7234740853309631, 1)],
  [(166, 197, 0.3185945451259613, 2)],
  [(157, 190, 0.8932272791862488, 3)],
  [(165, 254, 0.9302036762237549, 4)],
  [(164, 295, 0.7939915657043457, 5)],
  [(165, 190, 0.8248998522758484, 6)],
  [(165, 140, 0.509909451007843, 7)],
  [(165, 133, 0.7947704195976257, 8)],
  [(159, 93, 0.5575931668281555, 9)],
  [],
  [(190, 141, 0.8400986790657043, 10)],
  [(173, 125, 0.43527692556381226, 11), (165, 141, 0.3760838508605957, 12)],
  [(149, 133, 0.5660061836242676, 13)],
  [],
  []]]

In [9]:
str1 = ''.join(str(e) for e in nms_joint)

In [10]:
str1

'[[(157, 301, 0.35670003294944763, 0)], [(173, 254, 0.7234740853309631, 1)], [(166, 197, 0.3185945451259613, 2)], [(157, 190, 0.8932272791862488, 3)], [(165, 254, 0.9302036762237549, 4)], [(164, 295, 0.7939915657043457, 5)], [(165, 190, 0.8248998522758484, 6)], [(165, 140, 0.509909451007843, 7)], [(165, 133, 0.7947704195976257, 8)], [(159, 93, 0.5575931668281555, 9)], [], [(190, 141, 0.8400986790657043, 10)], [(173, 125, 0.43527692556381226, 11), (165, 141, 0.3760838508605957, 12)], [(149, 133, 0.5660061836242676, 13)], [], []]'

In [ ]:
json_nms_joint = dict()
with open('filename.json', 'w') as f:
    for i in range(len(nms_joint)):
        for j in range(16):
            if(len(nms_joint[i][j]) == 0):
                pass
            else:
                #print(j, nms_joint[i][j][0][0:2])
                json_nms_joint['%d'%j] = nms_joint[i][j][0][0:2]
        
        print(json_nms_joint)
        dumped = json.dumps(json_nms_joint, cls=NumpyEncoder)

        json.dump(dumped, f)
        f.write('\n')

        json_nms_joint.clear()
        print("------------------------------------")
        
    

In [3]:
v_list = os.listdir("./video_with_abstacle/TrainingSet_2/video_another_30/")

In [7]:
for i, d in enumerate (v_list):
    v_list[i] = "./video_with_abstacle/TrainingSet_2/video_another_30/" + d

In [16]:
file_name = 0
for index, data in enumerate (v_list):
    cap = cv2.VideoCapture(data)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    for i in range(length):
        ret, img = cap.read()
        if(ret == True):
            cv2.imwrite(("./video_with_abstacle/TrainingSet_2/img_another_30/" + "%d"%file_name + ".jpg"), img)
            file_name = file_name + 1
        else:
            continue
        